# Advent of Code

## 2022-012-016
## 2022 016

https://adventofcode.com/2022/day/16

In [3]:
import re
from collections import defaultdict
import heapq
import time

# Load and parse input file
file_path = 'input.txt'
with open(file_path, 'r') as file:
    data = file.readlines()

# Regex to parse each line
valve_pattern = re.compile(r"Valve (\w+) has flow rate=(\d+); tunnels? leads? to valves? (.+)")

# Create graph and flow rate dictionary
graph = defaultdict(list)
flow_rates = {}

for line in data:
    match = valve_pattern.match(line.strip())
    if match:
        valve, flow_rate, connections = match.groups()
        flow_rates[valve] = int(flow_rate)
        graph[valve].extend(connections.split(", "))

# Optimized A* implementation for the problem
def calculate_max_pressure_a_star_optimized(graph, flow_rates, start, max_time):
    # Priority queue: (-potential_pressure, -actual_pressure, current_valve, time_left, opened_valves)
    pq = [(-0, 0, start, max_time, frozenset())]
    visited = set()  # To avoid re-exploring identical states
    max_pressure = 0

    while pq:
        neg_potential, neg_actual, current, time_left, opened = heapq.heappop(pq)
        actual_pressure = -neg_actual

        # Track maximum pressure
        max_pressure = max(max_pressure, actual_pressure)

        # Skip if time runs out
        if time_left <= 0:
            continue

        # Avoid revisiting states
        state = (current, time_left, opened)
        if state in visited:
            continue
        visited.add(state)

        # Explore neighbors
        for neighbor in graph[current]:
            heapq.heappush(
                pq,
                (-actual_pressure, -actual_pressure, neighbor, time_left - 1, opened),
            )

        # Try opening the current valve if not already opened and it has a flow rate
        if current not in opened and flow_rates[current] > 0:
            new_pressure = actual_pressure + flow_rates[current] * (time_left - 1)
            potential_pressure = new_pressure + sum(
                flow_rates[v] * (time_left - 2) for v in flow_rates if v not in opened
            )  # Heuristic: include potential flow rates from other valves
            heapq.heappush(
                pq,
                (-potential_pressure, -new_pressure, current, time_left - 1, opened | {current}),
            )

    return max_pressure

# Measure timing
start_time = time.time()
start_valve = 'AA'
max_time = 30
max_pressure_a_star_optimized = calculate_max_pressure_a_star_optimized(
    graph, flow_rates, start_valve, max_time
)
end_time = time.time()

# Output result with timing
elapsed_time = end_time - start_time
formatted_time = f"{elapsed_time:.9f} s"
print(max_pressure_a_star_optimized, formatted_time)

2320 7.883614540 s


In [4]:
def calculate_max_pressure_with_elephant(graph, flow_rates, start, max_time):
    # Priority queue: (-potential_pressure, -actual_pressure, positions, time_left, opened_valves)
    pq = [(-0, 0, (start, start), max_time, frozenset())]
    visited = set()  # To avoid re-exploring identical states
    max_pressure = 0

    while pq:
        neg_potential, neg_actual, (pos1, pos2), time_left, opened = heapq.heappop(pq)
        actual_pressure = -neg_actual

        # Track maximum pressure
        max_pressure = max(max_pressure, actual_pressure)

        # Skip if time runs out
        if time_left <= 0:
            continue

        # Avoid revisiting states
        state = (pos1, pos2, time_left, opened)
        if state in visited:
            continue
        visited.add(state)

        # Generate actions for both actors (move or open a valve)
        for action1 in [(neighbor, False) for neighbor in graph[pos1]] + [(pos1, True)]:
            for action2 in [(neighbor, False) for neighbor in graph[pos2]] + [(pos2, True)]:
                # If both try to open the same valve, skip
                if action1[1] and action2[1] and action1[0] == action2[0]:
                    continue

                # Compute new state
                new_opened = set(opened)
                new_pressure = actual_pressure

                # Apply actions for actor 1
                if action1[1]:  # Open valve
                    if action1[0] not in new_opened and flow_rates[action1[0]] > 0:
                        new_opened.add(action1[0])
                        new_pressure += flow_rates[action1[0]] * (time_left - 1)

                # Apply actions for actor 2
                if action2[1]:  # Open valve
                    if action2[0] not in new_opened and flow_rates[action2[0]] > 0:
                        new_opened.add(action2[0])
                        new_pressure += flow_rates[action2[0]] * (time_left - 1)

                # Compute heuristic: prioritize remaining valves with high flow rates
                potential_pressure = new_pressure + sum(
                    flow_rates[v] * (time_left - 2)
                    for v in flow_rates if v not in new_opened
                )

                # Add to priority queue
                heapq.heappush(
                    pq,
                    (-potential_pressure, -new_pressure, (action1[0], action2[0]), time_left - 1, frozenset(new_opened)),
                )

    return max_pressure

# Measure timing
start_time = time.time()
max_time = 26
max_pressure_with_elephant = calculate_max_pressure_with_elephant(
    graph, flow_rates, start_valve, max_time
)
end_time = time.time()

# Output result with timing
elapsed_time = end_time - start_time
formatted_time = f"{elapsed_time:.9f} s"
max_pressure_with_elephant, formatted_time

KeyboardInterrupt: 

In [ ]:
import re
from collections import defaultdict
import heapq
import time

# Reload input file
file_path = 'input.txt'
with open(file_path, 'r') as file:
    data = file.readlines()

# Regex to parse each line
valve_pattern = re.compile(r"Valve (\w+) has flow rate=(\d+); tunnels? leads? to valves? (.+)")

# Create graph and flow rate dictionary
graph = defaultdict(list)
flow_rates = {}

for line in data:
    match = valve_pattern.match(line.strip())
    if match:
        valve, flow_rate, connections = match.groups()
        flow_rates[valve] = int(flow_rate)
        graph[valve].extend(connections.split(", "))

# Optimized A* implementation for two actors
def calculate_max_pressure_with_elephant_optimized(graph, flow_rates, start, max_time):
    """
    Optimized A* implementation for two actors working together to maximize pressure release.
    """
    # Priority queue: (-priority, -actual_pressure, (pos1, pos2), time_left, opened_valves)
    pq = [(-0, 0, (start, start), max_time, frozenset())]
    visited = set()  # To avoid re-exploring identical states
    max_pressure = 0

    def heuristic(opened, time_left):
        """
        Calculate the heuristic potential pressure from remaining unopened valves.
        """
        return sum(
            flow_rates[valve] * (time_left - 2)
            for valve in flow_rates if valve not in opened and time_left > 2
        )

    while pq:
        neg_priority, neg_actual, (pos1, pos2), time_left, opened = heapq.heappop(pq)
        actual_pressure = -neg_actual

        # Track maximum pressure
        max_pressure = max(max_pressure, actual_pressure)

        # Skip if time runs out
        if time_left <= 0:
            continue

        # Avoid revisiting states
        state = tuple(sorted((pos1, pos2))) + (time_left, frozenset(opened))
        if state in visited:
            continue
        visited.add(state)

        # Generate actions for both actors (move or open a valve)
        actions1 = [(neighbor, False) for neighbor in graph[pos1]] + [(pos1, True)]
        actions2 = [(neighbor, False) for neighbor in graph[pos2]] + [(pos2, True)]

        for action1 in actions1:
            for action2 in actions2:
                # Skip if both try to open the same valve
                if action1[1] and action2[1] and action1[0] == action2[0]:
                    continue

                # Compute new state
                new_opened = set(opened)
                new_pressure = actual_pressure

                # Apply actions for actor 1
                if action1[1]:  # Open valve
                    if action1[0] not in new_opened and flow_rates[action1[0]] > 0:
                        new_opened.add(action1[0])
                        new_pressure += flow_rates[action1[0]] * (time_left - 1)

                # Apply actions for actor 2
                if action2[1]:  # Open valve
                    if action2[0] not in new_opened and flow_rates[action2[0]] > 0:
                        new_opened.add(action2[0])
                        new_pressure += flow_rates[action2[0]] * (time_left - 1)

                # Calculate heuristic and priority
                potential_pressure = new_pressure + heuristic(new_opened, time_left - 1)
                priority = -potential_pressure

                # Add to priority queue
                heapq.heappush(
                    pq,
                    (priority, -new_pressure, (action1[0], action2[0]), time_left - 1, frozenset(new_opened)),
                )

    return max_pressure

# Measure timing
start_time = time.time()
start_valve = 'AA'
max_time = 26
max_pressure_with_elephant_optimized = calculate_max_pressure_with_elephant_optimized(
    graph, flow_rates, start_valve, max_time
)
end_time = time.time()

# Output result with timing
elapsed_time = end_time - start_time
formatted_time = f"{elapsed_time:.3f} {elapsed_time % 1:.3f} {elapsed_time % 0.001:.3f} s"
max_pressure_with_elephant_optimized, formatted_time